In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import BatchNormalization
from keras.models import Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
img_width, img_height = 150, 150
batch_size = 32
epochs = 5


train_data_dir = '/content/drive/MyDrive/Final year project/CNN_test/Cropped_Images/train'
test_data_dir = '/content/drive/MyDrive/Final year project/CNN_test/Cropped_Images/test'

In [ ]:
from keras.applications import vgg16

#base_model = tf.keras.applications.MobileNetV2(input_shape=(150, 150, 3),include_top=False,weights='imagenet')

base_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

for layer in base_model.layers:
    layer.trainable = False

58900480/58889256 [==============================] - 1s 0us/step


In [ ]:
def addTopOnNet(bottom_model, num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(64,activation='relu')(top_model)
    top_model = Dense(32,activation='relu')(top_model)
    top_model = Dense(16,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='sigmoid')(top_model)
    return top_model

In [ ]:
FC_Head = addTopOnNet(base_model, 1)

model = Model(inputs = base_model.input, outputs = FC_Head)

print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255)


test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 3951 images belonging to 2 classes.
Found 997 images belonging to 2 classes.


In [ ]:
nb_train_samples = 3951
nb_test_samples = 997

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_test_samples // batch_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/5
123/123 [==============================] - 1024s 8s/step - loss: 0.2990 - accuracy: 0.9018 - precision_1: 0.9068 - recall: 0.8952 - val_loss: 0.2489 - val_accuracy: 0.9123 - val_precision_1: 0.9976 - val_recall: 0.8273
Epoch 2/5
123/123 [==============================] - 26s 211ms/step - loss: 0.0795 - accuracy: 0.9765 - precision_1: 0.9834 - recall: 0.9694 - val_loss: 0.1632 - val_accuracy: 0.9476 - val_precision_1: 0.9890 - val_recall: 0.9054
Epoch 3/5
123/123 [==============================] - 26s 213ms/step - loss: 0.0430 - accuracy: 0.9885 - precision_1: 0.9903 - recall: 0.9867 - val_loss: 0.2029 - val_accuracy: 0.9315 - val_precision_1: 0.9886 - val_recall: 0.8735
Epoch 4/5
123/123 [==============================] - 26s 211ms/step - loss: 0.0220 - accuracy: 0.9939 - precision_1: 0.9959 - recall: 0.9918 - val_loss: 0.4503 - val_accuracy: 0.8236 - val_precision_1: 0.9939 - val_recall: 0.6533
Epoch 5/5
123/123 [==============================] - 26s 212ms/step - loss: 0.016